In [1]:
import boto3, re
from sagemaker import get_execution_role
role = get_execution_role()

In [2]:
!cat entry_point.py

###################################################################
##################### IMPORTS Pt.1 ###########################
###################################################################

# Import to list directories 
import os

print(os.getcwd())
print(os.listdir(os.getcwd()))

### import for time and time monitoring 
import time
## Imports for Metrics operations 
import numpy as np 
## Imports for dataframes and .csv managment
import pandas as pd 

# TensorFlow library  Imports
import tensorflow as tf
print("tf version: ", tf.__version__)


os.system('pip install swish-activation')


import swish_package
from swish_package import swish



# Keras (backended with Tensorflow) Imports
from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoi

In [8]:
import tarfile
with tarfile.open('model.tar.gz', mode='w:gz') as archive:
    archive.add('1', recursive=True)


In [6]:
import sagemaker
sagemaker_session = sagemaker.Session()
#inputs = sagemaker_session.upload_data(path='model.tar.gz', key_prefix='model')

In [7]:
from sagemaker.tensorflow.model import TensorFlowModel
sagemaker_model = TensorFlowModel(model_data = 's3://' + sagemaker_session.default_bucket() + '/model/model.tar.gz',
                                  role = role,
                                  framework_version = '2.3',
                                  entry_point = 'entry_point.py')

In [8]:
%%time
predictor = sagemaker_model.deploy(initial_instance_count=1,
                                   instance_type='ml.m4.xlarge')

update_endpoint is a no-op in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


-------------!CPU times: user 1min 30s, sys: 10.6 s, total: 1min 40s
Wall time: 8min 22s


In [10]:
!pip install transformers==2.10

  Using cached transformers-2.10.0-py3-none-any.whl (660 kB)
  Using cached tokenizers-0.7.0-cp36-cp36m-manylinux1_x86_64.whl (3.8 MB)
  Using cached dataclasses-0.8-py3-none-any.whl (19 kB)
Processing /home/ec2-user/.cache/pip/wheels/49/25/98/cdea9c79b2d9a22ccc59540b1784b67f06b633378e97f58da2/sacremoses-0.0.43-py3-none-any.whl
  Using cached regex-2020.11.13-cp36-cp36m-manylinux2014_x86_64.whl (723 kB)
  Using cached sentencepiece-0.1.94-cp36-cp36m-manylinux2014_x86_64.whl (1.1 MB)
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/amazonei_tensorflow2_p36/bin/python -m pip install --upgrade pip' command.


In [11]:
import os 

import transformers
from transformers.modeling_tf_distilbert import TFDistilBertModel
# TensorFlow library  Imports
import tensorflow as tf
# Keras (backended with Tensorflow) Imports
from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler
from tensorflow.keras.callbacks import CSVLogger

In [14]:
from transformers import DistilBertTokenizer
# Configuration
MAX_LEN = 192
#MODEL = 'jplu/tf-xlm-roberta-large'
#MODEL = 'roberta-base'
MODEL = 'distilbert-base-multilingual-cased'
EPOCHS = 10
tokenizer = DistilBertTokenizer.from_pretrained(MODEL)


In [15]:
def regular_encode(texts, tokenizer, maxlen=512):
    enc_di = tokenizer.batch_encode_plus(
        texts, 
        return_attention_masks=False, 
        return_token_type_ids=False,
        pad_to_max_length=True,
        max_length=maxlen
    )
    return np.array(enc_di['input_ids'])

In [97]:
import numpy as np 
D = np.array(["امشي لساني علي "])

In [98]:
x_test = regular_encode(D, tokenizer, maxlen=MAX_LEN)

In [99]:
predictor.predict(x_test)

{'predictions': [[0.138935536]]}

In [100]:
import pandas as pd 

In [91]:
test = pd.read_csv("test.csv")

In [92]:
test

,id,content,lang
0,0,Doctor Who adlı viki başlığına 12. doctor olar...,tr
1,1,"Вполне возможно, но я пока не вижу необходимо...",ru
2,2,"Quindi tu sei uno di quelli conservativi , ...",it
3,3,Malesef gerçekleştirilmedi ancak şöyle bir şey...,tr
4,4,:Resim:Seldabagcan.jpg resminde kaynak sorunu ...,tr
...,...,...,...
63807,63807,"No, non risponderò, come preannunciato. Prefer...",it
63808,63808,"Ciao, I tecnici della Wikimedia Foundation sta...",it
63809,63809,innnazitutto ti ringrazio per i ringraziamenti...,it
63810,63810,Kaç olumlu oy gerekiyor? Şu an 7 oldu. Hayır...,tr


In [93]:
test.lang.unique()

array(['tr', 'ru', 'it', 'fr', 'pt', 'es'], dtype=object)